# Building a Deep Network in PyTorch

### Prerequisite:

Download the data from [here](https://drive.google.com/file/d/1czcJcoG06uT7-xF2_3mr9uBV3qVVb6Tg/view)
and unzip it to `deeplearning_v2/dataset/dogs_and_cats/` folder.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from PIL import Image
from utdl.data import loader

ModuleNotFoundError: No module named 'utdl'

In [ ]:
def visualize_image(img):
    img = img.numpy()
    img = (img.transpose(1, 2, 0) * 255).astype(np.uint8)

    return Image.fromarray(img)


transform = loader.get_transform(resize=(32, 32))
input_size = 32 * 32 * 3
train_dataset = loader.get_dataset("dogs_and_cats", "train", transform=transform)

visualize_image(train_dataset[0][0])

In [ ]:
# Define a two-layer perceptron
class TwoLayerPerceptron(torch.nn.Module):
    def __init__(self, n_hidden=100):
        super().__init__()
        self.hidden = torch.nn.Linear(input_size, n_hidden)
        self.activation = torch.nn.ReLU()
        self.output = torch.nn.Linear(n_hidden, 1)

    def forward(self, x):
        return self.output(self.activation(self.hidden(x.view(-1))))

In [ ]:
# Let's create such a two-layer perceptron with 100 hidden units
net1 = TwoLayerPerceptron(100)
# We can pass an image through it and get the output logit
x, y = train_dataset[0]
print(f"net1 output: {net1(x).view(-1).detach().numpy()[0]} gt_label: {y}")
visualize_image(x)

In [ ]:
# Now let's pass another image through it and get the output logit
x, y = train_dataset[199]
print(f"net1 output: {net1(x).view(-1).detach().numpy()[0]} gt_label: {y}")
visualize_image(x)

In [ ]:
# When the network goes deeper, it can be painful to explicitly call each layer one by one.
# We can wrap layers into one module with a torch.nn.Sequential container
class MLP(torch.nn.Module):
    def __init__(self, input_size, *hidden_size):
        super().__init__()
        layers = []
        # Add hidden layers
        n_in = input_size
        for n_out in hidden_size:
            layers.append(torch.nn.Linear(n_in, n_out))
            layers.append(torch.nn.ReLU())
            n_in = n_out
        # Add the output layer
        layers.append(torch.nn.Linear(n_out, 1))

        # Use torch.nn.Sequential to create a small model,
        # where the layers are connected in a cascading way.
        # The order they are passed in the constructor
        self.network = torch.nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x.view(-1))

In [ ]:
# Let's create such a four-layer perceptron with 100, 50, 50 hidden units for each layer
net1 = MLP(input_size, 100, 50, 50)
# We can pass an image through it and get the output logit
x, y = train_dataset[0]
print(f"net1 output: {net1(x).view(-1).detach().numpy()[0]} gt_label: {y}")
visualize_image(x)